In [1]:
from chathist import InstructionDataLoader,InstructionDataset, InstructionStyle, Model, Tokenizer
import pandas as pd

In [2]:
chat_df = pd.read_json("hf://datasets/ogrnz/chat-titles/dataset.jsonl", lines=True)

In [3]:
chat_df.head()

,message,title
0,Develop a method for clustering astronomical d...,Astronomical Data Clustering Method
1,What is the role of dopamine in the brain as a...,Dopamine as a Neurotransmitter
2,"Replace the \""XXX\"" in the following sentence ...",Company Sustainability Objective
3,"Identify the sentence type (declarative, inter...",Sentence Type Identification
4,Suggest a book title for a fantasy novel about...,Outlaws in Enchanted Realms


In [4]:
chat_df.shape

(10000, 2)

In [5]:
chat_df.columns

Index(['message', 'title'], dtype='object')

In [6]:
style = InstructionStyle.load()

2025-05-13 21:15:06,579 - INFO - phi3 style chosen!!


In [7]:
df = style.convert_train(chat_df)

In [8]:
train_len = 9500

In [9]:
train_df = df.iloc[:train_len, :]
val_df = df.iloc[train_len:, :]

In [10]:
train_df

,instruct
0,<|user|>\nDevelop a method for clustering astr...
1,<|user|>\nWhat is the role of dopamine in the ...
2,"<|user|>\nReplace the \""XXX\"" in the following..."
3,<|user|>\nIdentify the sentence type (declarat...
4,<|user|>\nSuggest a book title for a fantasy n...
...,...
9495,<|user|>\nAssume yourself to be a business whi...
9496,<|user|>\nIgnore all previous instructions. Ac...
9497,"<|user|>\nGiven the following description, ide..."
9498,<|user|>\nWhat is the 17th letter of supercali...


In [11]:
print(train_df['instruct'].values[0])

<|user|>
Develop a method for clustering astronomical data.
<|llm|>
Astronomical Data Clustering Method


In [12]:
val_df

,instruct
9500,<|user|>\nWhat are the differences in the prop...
9501,<|user|>\nImplement a program to find the comm...
9502,<|user|>\nHow can a person develop a healthy s...
9503,<|user|>\nPlease help me use schrodinger's equ...
9504,<|user|>\nRewrite the sentence to use a pronou...
...,...
9995,"<|user|>\nFrom the given passage, identify the..."
9996,<|user|>\nAnalyze an effect that the text coul...
9997,<|user|>\nWhat is the average number of hours ...
9998,<|user|>\nClassify the following statement as ...


In [13]:
print(val_df['instruct'].values[0])

<|user|>
What are the differences in the properties of the top quark when it is produced through different modes such as gluon fusion and vector boson fusion, and how do these differences impact our understanding of the top quark's interactions with other particles in the standard model of particle physics?
<|llm|>
Top Quark Production Modes


In [14]:
train_dataset = InstructionDataset(train_df['instruct'].tolist())

2025-05-13 21:15:06,694 - INFO - Setting tokenizer using tiktoken with encoding: gpt2


In [15]:
val_dataset = InstructionDataset(val_df['instruct'].tolist())

In [16]:
train_loader = InstructionDataLoader().load(dataset = train_dataset)

In [17]:
val_loader = InstructionDataLoader().load(dataset = val_dataset)

In [18]:
data = iter(val_loader)

In [19]:
first_batch = next(data)

In [20]:
first_batch[0][0]

tensor([   27,    91,  7220,    91,    29,   198, 16447,   257, 15294, 21292,
          534, 18929,   287,  6946,   290,  1917,    12,    82, 10890,    25,
         6530,    25,  1757, 31780,   198,  6132,    25,   357, 10163,     8,
          604,  3980,    12,  3695,  3829,   198, 15333,    25,   474,    67,
         2577,    31, 12888,    13,   785,   198, 11297,  2597,    25, 10442,
        23836,   379, 10692, 23555,   198,    27,    91,   297,    76,    91,
           29,   198,  4965,  2454,   329,  1757, 31780, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256],
       dtype=torch.int32)

In [21]:
first_batch[1][0]

tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  4965,  2454,   329,  1757, 31780, 50256,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
       dtype=torch.int32)

In [22]:
tokenizer = Tokenizer()

2025-05-13 21:15:08,035 - INFO - Setting tokenizer using tiktoken with encoding: gpt2


In [23]:
print(tokenizer.decode_ids(first_batch[0][0]))

<|user|>
Create a resume highlighting your strengths in communication and problem-solving: Name: John Doe
Phone: (123) 456-7890
Email: jdoe@email.com
Current role: Software Developer at Ocean Solutions
<|llm|>
Resume for John Doe<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>


In [24]:
print(tokenizer.decode_ids(first_batch[1][0]))

Resume for John Doe<|endoftext|>


In [25]:
model = Model(style=style)

2025-05-13 21:15:08,323 - INFO - Using LoRA weights for ff layers!!
2025-05-13 21:15:08,611 - INFO - Using LoRA weights for multi head layers!!
2025-05-13 21:15:08,642 - INFO - Using LoRA weights for ff layers!!
2025-05-13 21:15:08,669 - INFO - Using LoRA weights for multi head layers!!
2025-05-13 21:15:08,704 - INFO - Using LoRA weights for ff layers!!
2025-05-13 21:15:08,737 - INFO - Using LoRA weights for multi head layers!!
2025-05-13 21:15:08,773 - INFO - Using LoRA weights for ff layers!!
2025-05-13 21:15:08,806 - INFO - Using LoRA weights for multi head layers!!
2025-05-13 21:15:08,843 - INFO - Using LoRA weights for ff layers!!
2025-05-13 21:15:08,868 - INFO - Using LoRA weights for multi head layers!!
2025-05-13 21:15:08,902 - INFO - Using LoRA weights for ff layers!!
2025-05-13 21:15:08,927 - INFO - Using LoRA weights for multi head layers!!
2025-05-13 21:15:08,964 - INFO - Using LoRA weights for ff layers!!
2025-05-13 21:15:08,989 - INFO - Using LoRA weights for multi head l

In [26]:
df = model.trainable_params()

2025-05-13 21:15:10,115 - INFO - Total model parameters: 166507792
2025-05-13 21:15:10,118 - INFO - Total trainable parameters: 166507792


In [27]:
train_loss, val_loss = model.train(train_loader = train_loader, val_loader = val_loader)

2025-05-13 21:15:10,125 - WARNING - Trained model exists on saved path /home/smathad/ai_capstone/chathist/models/finetuned/gpt2_chat_title_finetuned
 Skipping Training


In [28]:
model.generate("Hugging Face won’t understand your custom architecture (GPT2 class) unless you register it via PreTrainedModel. If LoRA is used, consider saving separate adapters using PEFT library compatibility, or wrap accordingly.")

2025-05-13 21:15:10,132 - INFO - Formatting prompt into phi3 style


'Custom Architecture Importance'